In [14]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate();
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark


In [15]:
import os
import tweepy as tw

In [16]:
access_token = '1423506172003291139-yQlvw5xX24vOmHSLCj1uKrYDuhQTKD'
access_token_secret = 'X2aRu8OMzNROksy4uIqLDf5hoMvXuR4xaEkYRYlyOxbdQ'
consumer_key = 'fRpq6Rk6LeDjJcq77AzR9vpFt'
consumer_secret = 'GeTDjPUQytrlKBx07KDqAKLTyRa7dfwP5JRqqKphmFujgx9Zx3'

In [17]:
auth = tw.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api= tw.API(auth,wait_on_rate_limit =True)

In [18]:
search_words = "#olympic"
date_since = "2021-08-07"

In [19]:
tweets = tw.Cursor(api.search,q=search_words,lang="en",since=date_since).items(200)
#tweets = tw.Cursor(api.search, lang="en",since=date_since).items(20)
[tweet.text for tweet in tweets]

['With 50 + startups attaining the unicorn status and with a new record for the most amount of medals won at the Olym… https://t.co/qrp92Wuzkc',
 'With 50 + startups attaining the unicorn status and with a new record for the most amount of medals won at the Olym… https://t.co/56ffJ9H0I0',
 "RT @APPOLICE100: It's more than a medal, it is India's pride: Andhra Pradesh Police felicitate @Pvsindhu1 for bagging the Olympic #Bronze M…",
 'Neeraj Incredible Chopra 🔥\nAn inspiration to stay desi...\nAnd never forget you mother tongue... #desi #NeerajChopra… https://t.co/IkHrCyeXqJ',
 'On the eve of #IndependenceDay President \nKovind to host #Indian #Olympic contingent over ‘high tea’… https://t.co/AWmvnZFOgz',
 'RT @16Sreejesh: The smile define everything 💪🙏\n\n@Neeraj_chopra1 \n#history #gold #athletics #olympic #tokyo https://t.co/4rCZku4vny',
 'RT @womensbadminton: 🏸🇮🇩🥇 Here’s my look at GreyAp’s amazing #Olympic campaign #badminton\n\nhttps://t.co/Vr7pqalq0c\n📷Rachel Florencia @persi…',
 

In [20]:
new_search= search_words 
new_search

'#olympic'

In [21]:

tweets = tw.Cursor(api.search, q=new_search,lang="en",since=date_since).items(50)
users_details = [[tweet.user.screen_name,tweet.user.followers_count,tweet.user.name,tweet.user.verified]for tweet in tweets]
users_details

[['teachedison', 125, 'TeachEdison', False],
 ['theedisonos', 14, 'EdisonOS', False],
 ['DevoteeYs', 1, 'Ys Jagan Devotee', False],
 ['neeraj_choyal', 31, 'Neeraj Choyal', False],
 ['RatnakarKrishn1', 411, 'Ratnakar', False],
 ['AkashMk24520486', 7, 'Akash Mk', False],
 ['AyuTriwn', 845, 'pengabdiwifi', False],
 ['missartola', 5367, 'smita mishra 🇮🇳', False],
 ['crownchronicles', 9770, 'The Crown Chronicles', False],
 ['CricGorgeous', 154, 'Mahi', False],
 ['BotMobilegame', 425, 'Mobile Game Bot', False],
 ['KishanMaiyad', 46, 'KRUSHNAM', False],
 ['MyLife_Congress', 3531, 'I Am Being Human 💕🖐💕', False],
 ['PavanKu19517349', 9, 'PAVAN KUMAR PRAJAPTI', False],
 ['chomsessentials', 57, 'Chomsessentials', False],
 ['RFYouthSports', 4919, 'RF Youth Sports', True],
 ['Nitish84984891', 4, 'Nitish', False],
 ['velvetangels17', 837, 'Sofia⁷≷ (fan.acc)🥰💫🍒⚾️🏐🏸', False],
 ['SalemDinesh12', 4, 'Salem Dinesh', False],
 ['JustAsk46082871', 41, 'Just Asking', False],
 ['GtwoRath', 232, 'Jitu', False]

In [22]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([ \
    StructField("screen_name",StringType(),True), \
    StructField("followers_count", IntegerType(), True), \
    StructField("name",StringType(),True), \
    StructField("verified", StringType(), True) \
  ])
 
df = spark.createDataFrame(data=users_details,schema=schema)
df.printSchema()
#df.show(truncate=False)
df.where(df.verified == "true").show()

root
 |-- screen_name: string (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- verified: string (nullable = true)

+-------------+---------------+--------------------+--------+
|  screen_name|followers_count|                name|verified|
+-------------+---------------+--------------------+--------+
|RFYouthSports|           4919|     RF Youth Sports|    true|
| CNBCTV18News|         956646|           CNBC-TV18|    true|
|      BiIndia|         110777|Business Insider ...|    true|
+-------------+---------------+--------------------+--------+

